In [1]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


In [2]:
# Smaller LLM, this is the one that we are trying to optimize for, the prompts are going to be tweaked
# to get the best out of this model
lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
#dspy.configure(lm=lm_gpt35)

# lm_gpt41mini = dspy.LM('gpt-4.1-mini', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
# lm_gpt4omini = dspy.LM('gpt-4o-mini', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)


# # Larger LLM, this is the one that we are going to use to optimize the prompts
# # It will be the helper/teach/AI Judge to assist in the optimization process
lm_gpt4 = dspy.LM('gpt-4.1', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4)

In [4]:
# Create domain classes
from typing import Literal
    
class NumberPicker(dspy.Signature):
    """Guess a number"""
    number_guess: str = dspy.InputField()
    answer: Literal["one", "two"] = dspy.OutputField()

numberPickerPredict = dspy.Predict(NumberPicker)
numberPickerPredict(number_guess="even")

Prediction(
    answer='two'
)

In [5]:
trainset = [
    dspy.Example(number_guess="1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="One", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Number one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Number two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Guess: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Guess: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="One", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="The number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Select the number: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Choose: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Pick 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Number one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Number two", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Option 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Answer is 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Digit: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 1", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Numeric value: 2", answer="two").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: one", answer="one").with_inputs("number_guess"),
    dspy.Example(number_guess="Textual value: two", answer="two").with_inputs("number_guess")
]


In [6]:
def validate_match(expected, actual, trace=None) -> bool:
    print()
    md("**expected**: ", expected)
    print("**actual**: ", actual)
    md("**Is match**: ", actual.answer == "two")
    print()
    return (actual.answer == "two")

from dspy.teleprompt import *

tp = dspy.MIPROv2(metric=validate_match, auto="heavy", prompt_model=lm_gpt35, task_model=lm_gpt4)
optimized_matcher = tp.compile(numberPickerPredict, trainset=trainset, requires_permission_to_run=False)

2025/09/04 14:52:51 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING HEAVY AUTO RUN SETTINGS:
num_trials: 27
minibatch: False
num_fewshot_candidates: 18
num_instruct_candidates: 9
valset size: 40

2025/09/04 14:52:51 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/09/04 14:52:51 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/09/04 14:52:51 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=18 sets of demonstrations...


Bootstrapping set 1/18
Bootstrapping set 2/18
Bootstrapping set 3/18


  0%|          | 0/10 [00:00<?, ?it/s]

**expected**: Example({'number_guess': '1', 'answer': 'one'}) (input_keys={'number_guess'})

**actual**:  Prediction(
    answer='one'
)


...

**actual**:  Prediction(
    answer='one'
)


**Is match**: False


Average Metric: 20.00 / 40 (50.0%): 100%|██████████| 40/40 [00:23<00:00,  1.72it/s]

2025/09/04 15:04:38 INFO dspy.evaluate.evaluate: Average Metric: 20 / 40 (50.0%)


2025/09/04 15:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.0 with parameters ['Predictor 0: Instruction 1', 'Predictor 0: Few-Shot Set 14'].
2025/09/04 15:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [50.0, 50.0, 50.0, 50.0, 47.5, 50.0, 50.0, 47.5, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 47.5, 47.5, 50.0, 50.0, 50.0, 50.0, 50.0, 47.5, 50.0, 50.0, 50.0, 50.0, 42.5, 50.0]
2025/09/04 15:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 50.0
2025/09/04 15:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/09/04 15:04:38 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 50.0!


In [ ]:
###########################################################################################

In [ ]:
optimized_matcher.save("./saved_files/", save_program=True)

In [19]:
optimized_matcher.save("./saved_files/test.json")

In [18]:
optimized_matcher(number_guess="The biggest numer")


Prediction(
    answer='two'
)

In [14]:

if hasattr(optimized_matcher, "teleprompter_state"):
    print("=== Teleprompter State ===")
    print(optimized_matcher.teleprompter_state)
else:
    print("No teleprompter_state found.")


No teleprompter_state found.


In [13]:
print("=== Examples in Optimized Matcher ===")
print(hasattr(optimized_matcher, "examples"))
if hasattr(optimized_matcher, "examples"):
    print(len(optimized_matcher.examples), " examples:")
    for ex in optimized_matcher.examples:
        print(ex)


=== Examples in Optimized Matcher ===
False


In [10]:

print("=== Signature ===")
print(getattr(optimized_matcher, "signature", None))

print("\n=== Examples ===")
if hasattr(optimized_matcher, "examples"):
    for ex in optimized_matcher.examples:
        print(ex)
else:
    print("No examples attribute found.")


=== Signature ===
NumberPicker(number_guess -> answer
    instructions='Guess a number'
    number_guess = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Number Guess:', 'desc': '${number_guess}'})
    answer = Field(annotation=Literal['one', 'two'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
)

=== Examples ===
No examples attribute found.


In [9]:
from dspy.teleprompt import render_prompt

# Render the optimized prompt for a sample input
sample_input = {"number_guess": "Pick 2"}
optimized_prompt_text = render_prompt(optimized_matcher, **sample_input)

print("=== Optimized Prompt ===")
print(optimized_prompt_text)


ImportError: cannot import name 'render_prompt' from 'dspy.teleprompt' (/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/teleprompt/__init__.py)

In [ ]:

optimized_prompt = getattr(optimized_matcher, "prompt", None) \
    or getattr(optimized_matcher, "instructions", None) \
    or getattr(optimized_matcher, "template", None)

print("**Optimized Prompt:**", optimized_prompt)

**Optimized Prompt:** None


In [ ]:
from dspy.teleprompt import render_prompt

print(render_prompt(optimized_matcher, number_guess="Pick 2"))


ImportError: cannot import name 'render_prompt' from 'dspy.teleprompt' (/usr/local/python/3.12.1/lib/python3.12/site-packages/dspy/teleprompt/__init__.py)

In [ ]:
if hasattr(optimized_matcher, "examples"):
    print("Few-shot examples:", optimized_matcher.examples)
elif hasattr(optimized_matcher, "demos"):
    print("Few-shot demos:", optimized_matcher.demos)

Few-shot demos: []


In [ ]:
print(getattr(optimized_matcher, "compiled_template", None))

None


In [ ]:

# Quick peeks (if available in your version)
try:
    print("=== optimized_matcher.inspect() ===")
    print(optimized_matcher.inspect())
except Exception:
    pass

try:
    print("=== tp.inspect() ===")
    print(tp.inspect())
except Exception:
    pass


=== optimized_matcher.inspect() ===
=== tp.inspect() ===


In [ ]:
from typing import Any

def _safe(obj: Any, attr: str, default=None):
    return getattr(obj, attr, default)

def _print_section(title: str, content):
    if content:
        print(f"\n===== {title} =====")
        print(content)

def print_dspy_program(program: Any):
    print(">>> Repr:", repr(program))

    # Signature + fields
    sig = _safe(program, "signature")
    _print_section("Signature", sig)
    if sig:
        # Some versions expose .instructions or a docstring
        _print_section("Signature Doc", getattr(sig, "__doc__", None))
        _print_section("Signature Instructions", _safe(sig, "instructions"))
        _print_section("Signature Inputs", _safe(sig, "inputs", _safe(sig, "input_fields", None)))
        _print_section("Signature Outputs", _safe(sig, "outputs", _safe(sig, "output_fields", None)))

    # Where different versions might store the optimized prompt/template
    for k in ("prompt", "instructions", "system_prompt", "template", "compiled_template"):
        _print_section(k, _safe(program, k))

    # Few-shot examples (these keys vary by version/teleprompter)
    examples = None
    for k in ("examples", "fewshot", "demos", "shots", "train_examples", "demo_store"):
        v = _safe(program, k)
        if v:
            _print_section(f"{k} (container)", f"type={type(v)}, len={getattr(v, '__len__', lambda: 'n/a')() if hasattr(v, '__len__') else 'n/a'}")
            examples = v
            break

    # Try to print examples if iterable
    if examples and hasattr(examples, "__iter__"):
        try:
            for i, ex in enumerate(list(examples)[:20]):  # cap to avoid huge logs
                print(f"\n--- Few-shot Example #{i+1} ---")
                if hasattr(ex, "inputs") and callable(ex.inputs):
                    print("inputs:", ex.inputs())
                elif hasattr(ex, "inputs"):
                    print("inputs:", ex.inputs)
                else:
                    print("inputs: <unknown>")

                if hasattr(ex, "outputs") and callable(ex.outputs):
                    print("outputs:", ex.outputs())
                elif hasattr(ex, "outputs"):
                    print("outputs:", ex.outputs)
                else:
                    print("outputs: <unknown>")
        except Exception as e:
            print(f"(could not iterate examples cleanly: {e})")

print("\n================= OPTIMIZED PROGRAM =================")
print_dspy_program(optimized_matcher)


================= OPTIMIZED PROGRAM =================
>>> Repr: Predict(NumberPicker(number_guess -> answer
    instructions='Guess a number'
    number_guess = Field(annotation=Literal['one', 'two'] required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Number Guess:', 'desc': '${number_guess}'})
    answer = Field(annotation=Literal['one', 'two'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

===== Signature =====
NumberPicker(number_guess -> answer
    instructions='Guess a number'
    number_guess = Field(annotation=Literal['one', 'two'] required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Number Guess:', 'desc': '${number_guess}'})
    answer = Field(annotation=Literal['one', 'two'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
)

===== Signature Doc =====
Guess a number

===== Signature Instructions =====
Guess a num